# Exploratory body landmark in oboe playign

Author: Kaitlin Pet 
2023


- Compare mutliple takes of oboe performance. Want to see if landmarks are similar in both takes 
- video orchestra folder

# RT demo from https://github.com/google/mediapipe/blob/master/docs/solutions/holistic.md
- stopping this will make kernel die! perform with care. 

In [ ]:

import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic


# For webcam input:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)
    print("image detected")
    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_hand_landmarks_style())

    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_hand_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      print("detection not working in jupyter")
      break
cap.release()

# Imports

Uncomment requirements to install

In [ ]:
!pip install librosa
#!pip install libfmp
!pip3 install mediapipe
!pip install opencv-python


In [ ]:
#indicates experiments being done on Kaitlin's computer vs server. Will consolidate version with nessesary files
local= True

In [ ]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
#import librosa
import cv2
#from google.colab.patches import cv2_imshow
import os
import pickle
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
import os
import pandas as pd
import time

In [ ]:
# for demo

view = 'front'
take = '1'

# Get alignment from orchestra

In [ ]:

perf_atm = os.path.join("./videos", view, view+"_take"+take, view+"_take"+take+'.atm')
perf_atm

In [ ]:
def get_alignment_from_orchestra(perf_atm, vid_frame_rate = 60):
    df = pd.read_csv(perf_atm, sep = r'\s{1,}', names= ['beat', 'time'])
    df = df.loc[df['time']!= 0]
    df['vidframe']= df['time']*vid_frame_rate
    return df

In [ ]:
note_position_df = get_alignment_from_orchestra(perf_atm)

# Hand detection alone#download off-the-shelf model

!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

## setup

In [ ]:
# Visualization function: shows how to yield values from fitted model
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.

  #print("length of identified landmarks: ", len(hand_landmarks_list))
  for idx in range(len(hand_landmarks_list)):

    #print("*****\n idx is ", idx)
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]
    #print( hand_landmarks)

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [ ]:
def get_hand_landmarks(src_color, detector, detector_type = 'video', timestamp=None, display=False):


    #make sure color space is correct before converting to mediapipe readable image type
    image= cv2.cvtColor(src_color, cv2.COLOR_BGR2RGB)
    
    if detector_type =='image':
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)

        # STEP 4: Detect hand landmarks from the input image.
        detection_result = detector.detect(mp_image)
    elif detector_type == 'video':

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
        detection_result = detector.detect_for_video(mp_image, timestamp)
    else:
        print("invalid detector type, should be \n\"image\"\n or \n\"video\"\n")
    # Extract normalized hand landmarks
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness

    normalized_hand_dictionary = {}
    for idx in range(len(hand_landmarks_list)):

      hand_landmarks = hand_landmarks_list[idx]
      allpoints = np.array([ np.array((landmark.y, landmark.x)) for landmark in hand_landmarks])

      #print(allpoints)
      normalized_hand_dictionary[handedness_list[idx][0].category_name]=allpoints


    # Transform nomalized hand landmarks into image coordinates
    hand_dictionary = {cat:normalized_hand_dictionary[cat] * (src_color.shape[0], src_color.shape[1])
                       for cat in normalized_hand_dictionary}

    # flip back coordinates: try to fix this more sustainably later
    hand_dictionary= {cat:np.flip(hand_dictionary[cat], axis=1) for cat in hand_dictionary}
    
    
    if display==True:
            
        plt.figure(figsize=(15, 15))
        plt.imshow(image, origin='lower')
        for idx in range(hand_dictionary['Left'].shape[0]):
            pointLeft = hand_dictionary['Left'][idx]
            pointRight = hand_dictionary['Right'][idx]
            plt.plot([pointLeft[0], pointRight[0]], [pointLeft[1], pointRight[1]], 'o',label=str(idx))
        plt.legend()
        plt.title("Hand landmark Legend")
        plt.show()
    return hand_dictionary, detection_result

In [ ]:
#download off-the-shelf model

!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

## run

In [ ]:
def get_rh_and_lh_landmarks(VID_FILE, disp_frames=500, show = True):
    # Use OpenCV’s VideoCapture to load the input video.
    cap = cv2.VideoCapture(VID_FILE)
    # Load the frame rate of the video using OpenCV’s CV_CAP_PROP_FPS
    # You’ll need it to calculate the timestamp for each frame.
    fps = cap.get(cv2.CAP_PROP_FPS)

    fpms = fps/1000
    print("frames per ms", fpms)
    total_num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("total frames in video", total_num_frames)


    ## Downsample
    ret = cap.set(cv2.CAP_PROP_FRAME_WIDTH,320)
    ret = cap.set(cv2.CAP_PROP_FRAME_HEIGHT,240)
    base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
    options = vision.HandLandmarkerOptions(base_options=base_options,
                                           running_mode=VisionRunningMode.VIDEO,
                                           num_hands=2)
    detector = vision.HandLandmarker.create_from_options(options)


    rh_res = []
    lh_res = []

    # Loop through each frame in the video using VideoCapture#read()
    cur_frame = 0
    
    time_start = time.time()
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == False:
            break
        frame.flags.writeable = False
        #need to convert to rgb before it can be processed
        frame= cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #downsample
        frame = cv2.pyrDown(frame)
       # print(frame.shape)


        frame_timestamp_ms = int(cur_frame/fpms)


        hand_dictionary, detection_result = get_hand_landmarks(frame, detector, 
                                   timestamp = frame_timestamp_ms,
                                  display=False)

        if cur_frame % disp_frames ==0:
            if show:
                frame.flags.writeable = True
                annotated_image = draw_landmarks_on_image(frame, detection_result)
                plt.figure(figsize=(20,10))
                plt.imshow(annotated_image)
                plt.show()
            print("Time elasped, ",  time.time() - time_start, "after  ", cur_frame*.033, "seconds in video elpased")
        if 'Right' in hand_dictionary:
            #Save hand landmark data        
            rh_res.append(hand_dictionary['Right'])

        else:
            rh_res.append([])

        if 'Left' in hand_dictionary:
            lh_res.append(hand_dictionary['Left'])      
        else:
            lh_res.append([])
        cur_frame = cur_frame + 1 #update

    cap.release()
    cv2.destroyAllWindows()
    
    return lh_res, rh_res

## analyze

- hand labeling is funky, try full body model

### functions

In [ ]:
def get_clean_points(hand_take):
    start = 0
    clean_data = []
    
    for i in range(len(hand_take)):
        el = hand_take[i]
 
        
        # video starts with no hands detected  
        if  len(el)==0 and len(clean_data)==0:

            start +=1   
        #middle of video with no hands detected 
        elif len(el)==0:
            clean_data.append(clean_data[-1])

        #normal case
        else:
            clean_data.append(el)

    return clean_data, start

In [ ]:
#takes a list of detection results(single hand), returns valid indices and points
#boundary frame is where one hand starts and other stops
def divide_data_clean(res, boundary_frame):
    
    take1 = res[0:boundary_frame]
    take2 = res[boundary_frame:len(res)]
    
    
    t1_clean_data, t1_start= get_clean_points(take1)
    t2_clean_data, t2_start = get_clean_points(take2)
    
    return {'t1':(t1_start, np.array(t1_clean_data)), 't2':(t2_start, np.array(t2_clean_data))}

### Front view (slightly right), RH only

need to do rh and lh separately

In [ ]:
VID_FILE = "./videos/schuman_1_frontview_2takes.MOV"

In [ ]:
boundary_frame = (60 + 23) * 30
boundary_frame

In [ ]:
rh_res, lh_res = get_rh_and_lh_landmarks(VID_FILE,disp_frames=50,  show=True)





In [ ]:
takes = divide_data_clean(lh_res, boundary_frame)

In [ ]:
rh_t1, rh_t2 = takes['t1'][1], takes['t2'][1]

In [ ]:
# normalize with wrist

In [ ]:
wrist_motion_t1  = rh_t1[:, 0, :]
wrist_motion_t2  = rh_t2[:, 0, :]

In [ ]:
plt.plot(wrist_motion_t1[:, 0] )
plt.plot(wrist_motion_t2[:, 0]-100 )

In [ ]:
plt.plot(wrist_motion_t1[:, 1] )
plt.plot(wrist_motion_t2[:, 1] -200)

In [ ]:
for i in range(5, rh_pnts.shape[1]):
    

    
    t1_landmark = rh_t1[:, i, :] - wrist_motion_t1
    t2_landmark = rh_t2[:, i, :] - wrist_motion_t2
    
    plt.figure(figsize=(15,10))
    plt.title("Landmark Y "+ str(i))
    plt.plot(t1_landmark[:,1] )
    plt.plot(t2_landmark[:,1]- 100 , alpha = .5)
   
    
    plt.show()
    
    plt.figure(figsize=(20,10))
    plt.title("Landmark X "+ str(i))
    plt.plot(t1_landmark[:,0] )
    plt.plot(t2_landmark[:,0]- 100 , alpha = .5)
   
    
    plt.show()

### Side view, RH only

In [ ]:
VID_FILE = "./videos/schuman_1_sideview.MOV"

In [ ]:
boundary_frame = (60 + 34) * 30
boundary_frame

In [ ]:
rh_res, lh_res = get_rh_and_lh_landmarks(VID_FILE)

In [ ]:
takes = divide_data_clean(lh_res, boundary_frame)

In [ ]:
rh_t1, rh_t2 = takes['t1'][1], takes['t2'][1]

In [ ]:
wrist_motion_t1  = rh_t1[:, 0, :]
wrist_motion_t2  = rh_t2[:, 0, :]

In [ ]:
plt.plot(wrist_motion_t1[:, 0] )
plt.plot(wrist_motion_t2[:, 0]-100 )

In [ ]:
plt.plot(wrist_motion_t1[:, 1] )
plt.plot(wrist_motion_t2[:, 1] -200)

In [ ]:
for i in range(5, rh_pnts.shape[1]):
    

    
    t1_landmark = rh_t1[:, i, :] - wrist_motion_t1
    t2_landmark = rh_t2[:, i, :] - wrist_motion_t2
    
    plt.figure(figsize=(15,10))
    plt.title("Landmark Y "+ str(i))
    plt.plot(t1_landmark[:,1] )
    plt.plot(t2_landmark[:,1]- 100 , alpha = .5)
   
    
    plt.show()
    
    plt.figure(figsize=(20,10))
    plt.title("Landmark X "+ str(i))
    plt.plot(t1_landmark[:,0] )
    plt.plot(t2_landmark[:,0]- 100 , alpha = .5)
   
    
    plt.show()

# Full body landmarks

In [ ]:
import mediapipe
mp_drawing = mediapipe.solutions.drawing_utils
mp_drawing_styles = mediapipe.solutions.drawing_styles
mp_holistic =mediapipe.solutions.holistic

In [ ]:
def get_generic_landmarks(landmarks):

  allpoints = np.array([ np.array((landmark.y, landmark.x)) for landmark in landmarks])

  return allpoints

In [ ]:
# this should be integrated into online pipeline but ok as separate function for now
def process_raw_landmarks(all_landmarks):
    all_pose_dct = {'RH':[],
                     'LH':[],
                     'Pose': [],
                     'Face': []}
    for timestep in all_landmarks:
        #print("*********")
        res_types = {'RH':timestep.right_hand_landmarks,
                     'LH':timestep.left_hand_landmarks,
                     'Pose': timestep.pose_landmarks,
                     'Face': timestep.face_landmarks
                    }
        for key in res_types:
            if res_types[key]== None:
                res_types[key]= []
            else:
                #print(len(res_types[key].landmark))
                res_types[key]= get_generic_landmarks(res_types[key].landmark)


        for key in all_pose_dct:
            all_pose_dct[key].append(res_types[key])


    all_pose_arrs = {}

    for key in all_pose_dct:
        data, start = get_clean_points(all_pose_dct[key])

        all_pose_arrs[key]= {'data': np.array(data), 'start':start}

    return all_pose_arrs

In [ ]:

def process_video_all_landmarks(VID_FILE, display_freq = 500, show= True, lim_len = None):
    # Use OpenCV’s VideoCapture to load the input video.
    cap = cv2.VideoCapture(VID_FILE )
    # Load the frame rate of the video using OpenCV’s CV_CAP_PROP_FPS
    # You’ll need it to calculate the timestamp for each frame.
    fps = cap.get(cv2.CAP_PROP_FPS)
    fpms = fps/1000
    print("frames per ms", fpms)
    total_num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("total frames in video", total_num_frames)
    ## get width and height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))




    all_landmarks = []


    idx = 0
    time_start = time.time()
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
      while idx < total_num_frames:

        if idx %100 ==0:
          print("progess:", idx/total_num_frames, 'on frame', idx)
        success, image = cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
          continue


        #visualize

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        #model_start = time.time()
        results = holistic.process(image)
        #print('model took', time.time()-model_start)
        all_landmarks.append(results)


        if idx% display_freq ==0:

            print("Time elasped, ",  time.time() - time_start, "after  ", idx/60, "seconds in video elpased")
            if show:
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(
                    image,
                    results.face_landmarks,
                    mp_holistic.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_holistic.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles
                    .get_default_pose_landmarks_style())
                mp_drawing.draw_landmarks(
                    image,
                    results.left_hand_landmarks,
                    mp_holistic.HAND_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles
                    .get_default_hand_landmarks_style())

                mp_drawing.draw_landmarks(
                    image,
                    results.right_hand_landmarks,
                    mp_holistic.HAND_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles
                    .get_default_hand_landmarks_style())
                #save
                plt.figure(figsize=(15, 15))
                plt.imshow(image)
                plt.show()



        if idx==lim_len:
            break
        idx = idx+1



    cap.release()

    return process_raw_landmarks(all_landmarks)

In [ ]:
def vis_landmarks_with_midi_1d(all_landmarks_list, #list of landmarks
                           note_times_df_lst=None, #when (in frames) each note occurs 
                           norm_landmark_list=[], #assume no normalization
                            vis_landmark_idx=0,
                            vis_landmark_dim = 1,
                            vis_landmark_name= 'RH',
                            colors = ['red', 'blue'],
                            label = 'RH wrist yloc',#interp this eventually, for now assume 2 takes
                            shift_del = 0, #we can alter y axis location of differt takes for smoother visualization
                            axis_change = 0,
                            text_shift = .01, 
                            label_text = True,
                            label_axis = True,
                            seq_pch = '.'
                           ):
    

    
    

    

        
    #plot landmark sequence
    for i in range(len(all_landmarks_list)):
        
    
        #visualization scheme: every landmark of one take is drawn first in the same color        
        plot_color = colors[i]
        all_landmarks_in_take = all_landmarks_list[i]
        
        #extract the sequence to plot
        seq = all_landmarks_in_take[vis_landmark_name]['data'][:, vis_landmark_idx, vis_landmark_dim]
            
        if len(norm_landmark_list) != 0:
            
            if len(seq)== len(norm_landmark_list[i]):
                seq = seq-norm_landmark_list[i]
            else:
                
                print("shape of seq", seq.shape)
                print("shape of norm landmarks", norm_landmark_list[i].shape)
                print("Check dims, there must be a one to one relation between normalization shift and data value")
                return
        
            
       
        
        #plot landmarks 
        seq = seq - axis_change
        xstart = all_landmarks_in_take[vis_landmark_name]['start']
        frame_seq = np.arange(0, len(seq))+xstart
        plt.plot(frame_seq, seq, seq_pch , label =  "Take "+str(i)+": "+label, color = plot_color)


        
        
        #plot note times:
        if note_times_df_lst != None:
            
            note_times_df = note_times_df_lst[i]
            
            

            #determine which notes to plot
            
            notestart_time = int(note_times_df.iloc[0]['vidframe'])
            print(notestart_time)
            
 
            noteend_time = xstart + len(seq)
            print(noteend_time)
            
            all_note_times = note_times_df['vidframe']
            selected_note_times =all_note_times[all_note_times < noteend_time]
            
            
            marker_min_height = min(seq[notestart_time: noteend_time])- 2*text_shift
            marker_max_height  = max(seq[notestart_time: noteend_time])+2*text_shift
        

            if label_axis:
                print("axis labeled, i is ", i)
                plt.vlines(selected_note_times, marker_min_height, 
                       marker_max_height, color = plot_color, label = "Take "+str(i)+": note onset times")
            else:
                plt.vlines(selected_note_times, marker_min_height, 
                       marker_max_height, color = plot_color)


            #label with note index
            if label_text:
                for j in range(0, len(selected_note_times)):#len(note_times_df['vidframe'])):

                    #print(note_times_df['vidframe'][j])
                    plt.text( selected_note_times[j], marker_min_height-textshift,  str(j), color = plot_color)

    
    
        axis_change = axis_change - shift_del

    plt.legend()

In [ ]:
view = 'front'
take = '1'
VID_FILE1 = os.path.join("./videos", view, view+"_take"+take, view+"_take"+take+'.mov')
perf_atm1 = os.path.join("./videos", view, view+"_take"+take, view+"_take"+take+'.atm')

note_position_df1 = get_alignment_from_orchestra(perf_atm1)

In [ ]:
processed_landmarks_1 = process_video_all_landmarks(VID_FILE1, display_freq = 1000, show= True, lim_len = 1000)

In [ ]:

take = '2'
VID_FILE2 = os.path.join("./videos", view, view+"_take"+take, view+"_take"+take+'.mov')
print("file path:", VID_FILE2)
perf_atm2 = os.path.join("./videos", view, view+"_take"+take, view+"_take"+take+'.atm')

note_position_df2 = get_alignment_from_orchestra(perf_atm2)

In [ ]:
processed_landmarks_2 = process_video_all_landmarks(VID_FILE2, display_freq = 1000, show= True, lim_len = 1000)

In [ ]:
                           norm_landmark_list=[], #assume no normalization
                            vis_landmark_idx=0,
                            vis_landmark_dim = 1,
                            vis_landmark_name= 'RH',

In [ ]:
plt.figure(figsize=(20,10))
vis_landmarks_with_midi_1d([processed_landmarks_1, processed_landmarks_2], [note_position_df1, note_position_df2],
                           vis_landmark_name = 'LH', shift_del = .12,  label= "LH wrist yloc")

#vis_landmarks_with_midi_1d([processed_landmarks_1, processed_landmarks_2], [note_position_df1, note_position_df2],
#                           vis_landmark_name = 'RH', shift_del = .12,  label_text = True,
#                            label_axis = False, seq_pch = '+', axis_change = .1, label= "RH wrist yloc")

plt.ylim(.38, .6)
plt.xlim(200, None)

plt.title("Left Hand Wrist Motion")
plt.savefig("left_hand_wrist_schuman1_front.jpg")

In [ ]:
multi_take_landmarks = [processed_landmarks_1, processed_landmarks_2]
LH_wrist_x = [take['LH']['data'][:, 0, 0] for take in multi_take_landmarks]
LH_wrist_y = [take['LH']['data'][:, 0, 1] for take in multi_take_landmarks]
RH_wrist_x = [take['RH']['data'][:, 0, 0] for take in multi_take_landmarks]
RH_wrist_y = [take['RH']['data'][:, 0, 1] for take in multi_take_landmarks]

In [ ]:

plt.figure(figsize=(20,10))
vis_landmarks_with_midi_1d(multi_take_landmarks, 
                           [note_position_df1, note_position_df2],
                           LH_wrist_y,
                            vis_landmark_idx=20,
                            vis_landmark_dim = 1,
                           vis_landmark_name = 'LH', 
                           shift_del = .12, 
                           label = 'LH pinky yloc')

#vis_landmarks_with_midi_1d([processed_landmarks_1, processed_landmarks_2], [note_position_df1, note_position_df2],
#                           vis_landmark_name = 'RH', shift_del = .12,  label_text = True,
#                            label_axis = False, seq_pch = '+', axis_change = .1)

plt.ylim(-.12, .07)

plt.title("Left Hand Pinky fingertip - Y location\n(Normalized by left wrist y position)")
plt.xlim(200, None)

plt.savefig("left_hand_pinky_tip_yloc_schuman1_front.jpg")

In [ ]:

plt.figure(figsize=(20,10))
vis_landmarks_with_midi_1d(multi_take_landmarks, 
                           [note_position_df1, note_position_df2],
                           LH_wrist_x,
                            vis_landmark_idx=8,
                            vis_landmark_dim = 0,
                           vis_landmark_name = 'LH', shift_del = .12, label = 'LH middle fingertip xloc')

#vis_landmarks_with_midi_1d([processed_landmarks_1, processed_landmarks_2], [note_position_df1, note_position_df2],
#                           vis_landmark_name = 'RH', shift_del = .12,  label_text = True,
#                            label_axis = False, seq_pch = '+', axis_change = .1)

plt.ylim(-.155, .07)

plt.title("Left Hand middle fingertip - X location\nNormalized by wrist")
plt.xlim(200, None)

plt.savefig("left_hand_middle_fingertip_schuman1_front.jpg")

In [ ]:

plt.figure(figsize=(20,10))
vis_landmarks_with_midi_1d(multi_take_landmarks, 
                           [note_position_df1, note_position_df2],
                            vis_landmark_idx=0,
                            vis_landmark_dim = 1,
                           vis_landmark_name = 'RH', shift_del = .12, label = 'Nose xloc')

#vis_landmarks_with_midi_1d([processed_landmarks_1, processed_landmarks_2], [note_position_df1, note_position_df2],
#                           vis_landmark_name = 'RH', shift_del = .12,  label_text = True,
#                            label_axis = False, seq_pch = '+', axis_change = .1)

plt.ylim(.3, .6)

#plt.title("Left Hand middle fingertip - X location\nNormalized by wrist")
#plt.xlim(200, None)

#plt.savefig("left_hand_middle_fingertip_schuman1_front.jpg")

In [ ]:
plt.figure(figsize=(20,10))
vis_landmarks_with_midi_1d([processed_landmarks_1, processed_landmarks_2], 
                           vis_landmark_name = 'LH', shift_del = .1)


note_times_df = note_position_df1

marker_min = .4
marker_max = .5
plot_color = 'red'
textshift = .01

plot_notes = range(0,20)

plt.vlines(note_times_df['vidframe'], marker_min, marker_max, color = plot_color)
for j in plot_notes:#len(note_times_df['vidframe'])):
    #print(note_times_df['vidframe'][j])
    plt.text( note_times_df['vidframe'][j], marker_min-textshift,  str(j), color = plot_color)


note_times_df = note_position_df2

marker_min = .5
marker_max = .6
plot_color = 'blue'

plt.vlines(note_times_df['vidframe'], marker_min, marker_max, color = plot_color)
for j in plot_notes:#len(note_times_df['vidframe'])):
    #print(note_times_df['vidframe'][j])
    plt.text( note_times_df['vidframe'][j], marker_min-textshift,  str(j), color = plot_color)

plt.xlim(200, 1000)
plt.ylim(.35, .7)

In [ ]:
note_position_df1

In [ ]:
processed_landmarks_2['RH']['data'][:, 4, 0]

# Functions

Run this first

## Get frame alignment

In [ ]:
if not local:
    vid_pth = "/home/intern-2023-kaitlin/data/dataset/burgmuller-performances-from-top/b-01.mp4"
else:
    vid_pth = "../data/dataset/testing/b-01.mp4"



In [ ]:
def get_boundary_frame(vid_pth, time_of_boundary, time_of_ref): #time of frame in seconds
    cap = cv2.VideoCapture(vid_pth)
    total_num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(total_num_frames)
    boundary_frame = int(time_of_boundary*fps)
    ref_hand_frame = int(time_of_ref*fps)
    print(boundary_frame )
    i = 0
    while i < boundary_frame :
        ret, src_color = cap.read()
        
        if i== ref_hand_frame:
            hand_ref_frame = src_color.copy()
            plt.title("frame used for identifying hand positions at frame:"+str(i))
            plt.imshow(prepare_frame(hand_ref_frame), origin='lower')
            plt.show()
        i=i+1
    plt.title("Boundary frame "+str(boundary_frame))
    plt.imshow(prepare_frame(src_color), origin='lower')
    plt.show()
    alignment_computation_frame = src_color.copy()   
    cap.release()
    
    
    return hand_ref_frame, alignment_computation_frame, boundary_frame, fps

## Functions to obtain warp

- Inputs: 
    - path to template (should be hardcoded)
    - path to frame (in video would be inputted differently)
    - detector (mediapipe detector object, created above)
    - middle finger notes - Notes being played by the 2 middle fingers. This could be extracted automatically, but in practice probably easier as a user directive (i.e. play these notes with these fingers to initialize system)
    - display - boolean for visual output

In [ ]:
# From align_movie.ipynb notebook
def image_preprocessing_pipeline(img):    
    '''画像の前処理． グレースケールに変換する. できるだけ２つの画像の相関を高めるような操作にする．
    今回の場合，鍵盤の白黒がはっきりしているから，白黒を見るだけで相関が高くなると予想．'''
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    normalizedImg = np.zeros(img.shape)
    normalizedImg = cv2.normalize(img,  normalizedImg, 0, 255, cv2.NORM_MINMAX)
    img = normalizedImg
    return img


In [ ]:
def prepare_frame(src_color):
    
    return cv2.flip(src_color, 1)

In [ ]:

'''
template_color: template of piano (file provided)
src_color: image of hands on piano- used to find hand landmarks (should be start frame of original video)
middle_finger_notes: the notes played by middle fingers on each hand in src_color
detector: mediapipe hand landmarker detector
alignment_computation_frame: image of pinao used for template matching ( a different frame from origianl video)
'''
def get_piano_alignment_transform(template_color, src_color, 
                                  alignment_computation_frame ,
                                  detector, middle_finger_notes, 
                                  display=False, prevComputedInitialization = []):
    '''
    Preprocessing (rotate image to unified perspective of person's view of piano while playing)  
    '''
    src_color_display = prepare_frame(cv2.cvtColor(src_color, cv2.COLOR_BGR2RGB))
    src_color = prepare_frame(src_color)
    alignment_computation_frame = prepare_frame(alignment_computation_frame)
    
    '''
    Identify hand positions
    '''
    hand_dictionary = get_hand_landmarks(src_color, detector, 
                                         detector_type = 'image', 
                                         display=display)
    
    #locations of middle fingertips
    rh_middle_fingertip = hand_dictionary["Right"][MIDDLE_FINGERTIP_IDX]
    lh_middle_fingertip = hand_dictionary["Left"][MIDDLE_FINGERTIP_IDX]
    print("fingertip absolute locations", rh_middle_fingertip, lh_middle_fingertip)
    #notes being played (unit, position on template). WE ASSUME THIS IS CLOSE TO WHITE-BLACK KEY BOUNDARY
    rh_middle_finger_notepos = middle_finger_notes[1]*T_WIDTH/WHITE_KEYS_ON_PIANO
    lh_middle_finger_notepos = middle_finger_notes[0]*T_WIDTH/WHITE_KEYS_ON_PIANO
    print("fingertip positions on template", rh_middle_finger_notepos, lh_middle_finger_notepos)
    
    '''
    Extract score-based keypoints in image
    '''
    middle_finger_interval = middle_finger_notes[1] - middle_finger_notes[0]
    print("middle finger interval is ", middle_finger_interval)
    middle_finger_dist = np.sqrt(np.sum(lh_middle_fingertip-rh_middle_fingertip)**2)
    print("middle finger distance in image is ", middle_finger_dist)    
    
    '''
    Assign corresponding keypoints in template
    '''
    
    template_lh_middle_fingertip = np.array([lh_middle_finger_notepos, BLACK_KEY_BOTTOM_RATIO*T_HEIGHT])
    template_rh_middle_fingertip = np.array([rh_middle_finger_notepos, BLACK_KEY_BOTTOM_RATIO*T_HEIGHT])
    
    
    '''
    Assign Middle C position on template (bottom and top of note
    '''
    midpoint_note_pose = int(middle_finger_notes[0]+middle_finger_notes[1])/2+0.5
    print("new midpoint:", midpoint_note_pose)
    template_middle_c_top = np.array([midpoint_note_pose/WHITE_KEYS_ON_PIANO*T_WIDTH, T_HEIGHT])
    template_middle_c_bottom = np.array([midpoint_note_pose/WHITE_KEYS_ON_PIANO*T_WIDTH, 0] )

    
    '''
    Estimate length and width of piano
    '''
    src_piano_length_estimate = middle_finger_dist * WHITE_KEYS_ON_PIANO /middle_finger_interval
    print("piano length estimate", src_piano_length_estimate )
    src_piano_width_estimate = src_piano_length_estimate * T_HEIGHT/T_WIDTH
    print("piano width estimate", src_piano_width_estimate)
    black_key_height_estimate = (1-BLACK_KEY_BOTTOM_RATIO)*src_piano_width_estimate
    under_black_key_height_estimate = BLACK_KEY_BOTTOM_RATIO*src_piano_width_estimate
    
    '''
    Extract expected position of Middle C on image
    '''
    unit_middle_finger_vector = (rh_middle_fingertip-lh_middle_fingertip)/middle_finger_dist
    print("middle finger unit vecotr", unit_middle_finger_vector)
    c_from_left_interval =  midpoint_note_pose - middle_finger_notes[0]
    print("c_from_left_interval", c_from_left_interval)
    c_from_left_dst = middle_finger_dist*c_from_left_interval/middle_finger_interval
    print("distance of middle c from left finger is", c_from_left_dst)
    #estimate location of middle c by extending vector connecting two middle fingertips
    middle_c_blackey_edge = lh_middle_fingertip+ c_from_left_dst*unit_middle_finger_vector
    #use tangent vector to estimate locations of top and bottom of key
    unit_middle_finger_tangent_vector = np.array([unit_middle_finger_vector[1], 
                                              -1* unit_middle_finger_vector[0]])
    middle_c_top = middle_c_blackey_edge - black_key_height_estimate * unit_middle_finger_tangent_vector
    middle_c_bottom = middle_c_blackey_edge +under_black_key_height_estimate*unit_middle_finger_tangent_vector
    
    '''
    Warp from image to template
    '''
    
    #image
    x_tgt = np.array([rh_middle_fingertip[0], middle_c_top[0], lh_middle_fingertip[0], middle_c_bottom[0]])
    y_tgt = np.array([rh_middle_fingertip[1], middle_c_top[1], lh_middle_fingertip[1], middle_c_bottom[1]])
    Xt = np.c_[x_tgt,y_tgt].astype('float32')
    
    #template
    x_ref = np.array([template_rh_middle_fingertip[0], template_middle_c_top[0], template_lh_middle_fingertip[0], template_middle_c_bottom[0]])
    y_ref = np.array([template_rh_middle_fingertip[1], template_middle_c_top[1], template_lh_middle_fingertip[1], template_middle_c_bottom[1]])
    Xr = np.c_[x_ref,y_ref].astype('float32')    


    
    
    # Use the above info to find the transform matrix

    if len(prevComputedInitialization)>0:
        warp_initial = prevComputedInitialization
    else:
        warp_initial = cv2.getPerspectiveTransform(Xt, Xr).astype('float32')


    
    '''
    Visualize (sanity check)
    '''
    if display:
        #image keypoints
        plt.imshow(src_color_display, origin='lower')
        plt.plot(x_tgt, y_tgt, 'r')
        plt.title("keypoints on frame")
        plt.show()
        
        #template keypoints
        plt.imshow(template_color, origin='lower')
        plt.plot(x_ref, y_ref, 'r')
        plt.title("keyframes on template")
        plt.show()
        
        #initial warp alignment, SHOULD BE PRETTY CLOSE
        initial_warped_frame =  cv2.warpPerspective(src_color, 
                                            warp_initial, (src_color.shape[1], src_color.shape[0]))
        plt.imshow(template_color, alpha=.5, origin='lower')

        plt.imshow(initial_warped_frame, alpha = .5, origin='lower')
        plt.xlim(0, T_WIDTH)
        plt.ylim(0, T_HEIGHT)
        plt.title("Initial alignment, SHOULD BE CLOSE")
        plt.show()
        
    '''
    Refine with findECC
    '''
    
    #create mask for template(I think this makes id easier)
    t_mask = np.zeros((template_color.shape[0], template_color.shape[1]))
    
    #versions
    t_mask[:, int(template_lh_middle_fingertip[0])-200:int(template_rh_middle_fingertip[0])+200]=1
    #t_mask[:, int(template_lh_middle_fingertip[0]):int(template_rh_middle_fingertip[0])]=1    
    t_mask[:, int(template_lh_middle_fingertip[0]-200):]=1     
    
    t_mask= t_mask.astype(np.uint8)
    if display:
        plt.imshow(t_mask*image_preprocessing_pipeline(template_color), origin='lower')
        plt.show()
    
    (retval, warpMatrix) = cv2.findTransformECC(image_preprocessing_pipeline(alignment_computation_frame),
                                            image_preprocessing_pipeline(template_color),
                                           warp_initial  , cv2.MOTION_HOMOGRAPHY,
                                            criteria, inputMask=t_mask, gaussFiltSize=5)## GOT RID OF MASK
    
    if display:
        img2w = cv2.warpPerspective(src_color, warpMatrix, (T_WIDTH, T_HEIGHT))
        img2w = cv2.warpPerspective(src_color, warpMatrix, (src_color.shape[1], src_color.shape[0]))
        
        plt.imshow(img2w, origin='lower')
        plt.show()
        plt.imshow(img2w, origin='lower')
        plt.imshow(template_color, origin='lower', alpha=0.4)
        plt.title("Final alignment of piano")
        plt.show()
        print("in get matrix, final_wrp is \n", warpMatrix)        
    
    return warpMatrix

In [ ]:
def get_hand_landmarks(src_color, detector, detector_type = 'video', timestamp=None, display=False):


    #make sure color space is correct before converting to mediapipe readable image type
    image= cv2.cvtColor(src_color, cv2.COLOR_BGR2RGB)
    
    if detector_type =='image':
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)

        # STEP 4: Detect hand landmarks from the input image.
        detection_result = detector.detect(mp_image)
    elif detector_type == 'video':

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
        detection_result = detector.detect_for_video(mp_image, timestamp)
    else:
        print("invalid detector type, should be \n\"image\"\n or \n\"video\"\n")
    # Extract normalized hand landmarks
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness

    normalized_hand_dictionary = {}
    for idx in range(len(hand_landmarks_list)):

      hand_landmarks = hand_landmarks_list[idx]
      allpoints = np.array([ np.array((landmark.y, landmark.x)) for landmark in hand_landmarks])

      #print(allpoints)
      normalized_hand_dictionary[handedness_list[idx][0].category_name]=allpoints


    # Transform nomalized hand landmarks into image coordinates
    hand_dictionary = {cat:normalized_hand_dictionary[cat] * (src_color.shape[0], src_color.shape[1])
                       for cat in normalized_hand_dictionary}

    # flip back coordinates: try to fix this more sustainably later
    hand_dictionary= {cat:np.flip(hand_dictionary[cat], axis=1) for cat in hand_dictionary}
    
    
    if display==True:
            
        plt.figure(figsize=(15, 15))
        plt.imshow(image, origin='lower')
        for idx in range(hand_dictionary['Left'].shape[0]):
            pointLeft = hand_dictionary['Left'][idx]
            pointRight = hand_dictionary['Right'][idx]
            plt.plot([pointLeft[0], pointRight[0]], [pointLeft[1], pointRight[1]], 'o',label=str(idx))
        plt.legend()
        plt.title("Hand landmark Legend")
        plt.show()
    return hand_dictionary

## Functions for applying warping

- assume warpPerspective (4 points)


In [ ]:
def perform_warp(src,warp_mat, display=False):
    #Warp image

    #print("in compute perform warp, final_wrp is \n", warp_mat)
    warp_dst = cv2.warpPerspective(src, warp_mat, ( src.shape[1], src.shape[0]))


    if display:
            
        plt.imshow(warp_dst, origin='lower')
        plt.title("Warped image")
        plt.show()
    return warp_dst

WarpAffine can be performed with matrix multiplication, but warp perspective is harder to express using matrix algebra. I should look into this though, it will help any type of optimization

- Need to add extra dimension for bias in point terms: https://theailearner.com/tag/cv2-getaffinetransform/
- From the documentation here(https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87), the warpPerspective() matrix is applied thus:
    $$dst(x,y) = src(\frac{M_{11}x +M_{12}y+M_{13}}{M_{31}x+M_{32}y+M_{33}}, 
    \frac{M_{21}x+M_{22}y+M_{23}}{M_{31}x+M_{32}y+M_{33}})$$
    

In [ ]:
#assume input format of hand array is 

'''

[[key1x, key1y],
 [key2x, key2y]
 ...]

'''
def manual_warp_points(point_arr, M):

    #print("in manual warp points, final_wrp is \n", M)
    x = point_arr[:,0:1]
    y = point_arr[:,1:2]
    #print("stophere, ", point_arr)
    norm_factor = M[2,0]*x + M[2,1]*y+M[2,2]

    ret_x = M[0,0]*x + M[0,1]*y + M[0,2]
    ret_y = M[1,0]*x + M[1,1]*y+ M[1,2]

    non_norm_warped_points = np.concatenate((ret_x, ret_y), axis=1)

    warped_points = (non_norm_warped_points/norm_factor).T
    
    return warped_points
    
    

In [ ]:
def compute_transformed_hand_points(hand_dictionary, warp_mat, src_color, display=False):
    transformed_hand_points = {cat:manual_warp_points(hand_dictionary[cat], warp_mat) 
                              for cat in hand_dictionary}
    
    
    #print("in compute transformed points, final_wrp is \n", warp_mat)
    if display:
        warp_dst = perform_warp(src_color,warp_mat, display=False)
        plt.imshow(warp_dst, origin='lower')

        for cat in transformed_hand_points:
            cat_points =  transformed_hand_points[cat]
            plt.plot(cat_points[0], cat_points[1], 'r.', alpha=0.5)
        plt.title("Location hand keypoints after transformation")

        plt.show()
    
    return transformed_hand_points

## Function to Extract transformed hand points

In [ ]:
if not local:
    video_pth = "/home/intern-2023-kaitlin/data/dataset/burgmuller-performances-from-top/b-01.mp4"
else:
    video_pth = "../data/dataset/testing/b-01.mp4"

### Visualization parameters

In [ ]:
#vis parameters
markersize = 5
rh_keypoint_color = np.array([0, 0,255])
lh_keypoint_color = np.array([0, 0, 255])

rh_cloud_color = np.array([0, 100, 0]).astype(np.uint8)
lh_cloud_color = np.array([0, 100,255]).astype(np.uint8)

obs_idx = [0,12]

In [ ]:
saveVid=True

In [ ]:
def draw_notecloud(warp_dst, transformed_hand_dct, hand_cat, color):
    bounds = (np.min(transformed_hand_dct[hand_cat][0, :]), 
                         np.max(transformed_hand_dct[hand_cat][0, :]))
    warp_dst[:,int(bounds[0]): int(bounds[1]), :]+=color
    
    return warp_dst

In [ ]:
def draw_handpoints(warp_dst, transformed_hand_dct,  hand_cat, obs_idx, color):

    #Draw keypoints
    reduced_landmarks = transformed_hand_dct[hand_cat][:,obs_idx ].T
    #print(reduced_landmarks)
    
    for landmark in reduced_landmarks:
        x = int(landmark[0])
        y=int(landmark[1])

        #print("landmark is", landmark, "poisiton", x,y)
        warp_dst[(y-markersize):(y+markersize), (x-markersize):(x+markersize), :]=color          

    return warp_dst
        

### Function for extracting info

In [ ]:
def extract_hand_info_from_burgmuller(vid_pth, boundary_frame, 
                                      landmarker_model_pth, final_wrp,
                                      saveVid = False, outfile_stem = None, display=False):

    '''
    Video detector. Defining here for ease of debugging (it keeps an index of current progress)

    '''
    
    #shift image up
    warp_mat = final_wrp.copy()
    warp_mat[1,2]+= 300
    
    
    base_options = python.BaseOptions(model_asset_path=landmarker_model_pth)
    options = vision.HandLandmarkerOptions(base_options=base_options,  
                                           running_mode=VisionRunningMode.VIDEO,
                                           num_hands=2)
    video_detector = vision.HandLandmarker.create_from_options(options)

    '''
    Input video
    '''
    cap = cv2.VideoCapture(vid_pth)
    fps = cap.get(cv2.CAP_PROP_FPS)
    fpms = fps/1000 #frames per ms
    print("frames per ms", fpms)
    total_num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("total frames in video", total_num_frames)
    ## get width and height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    
    
    # Loop through each frame in the video using VideoCapture#read()
    cur_frame = 0

    '''
    Output video to see hand bounds
    Make first video now
    '''
    if saveVid:
        out1_pth = outfile_stem+"-1.avi"
        print("path of first video file is",out1_pth)
        out1 = cv2.VideoWriter(out1_pth,
                            cv2.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width, frame_height))


    #Save bounds
    lh_res=[]

    rh_res =[]

    while  cur_frame < total_num_frames:


        if cur_frame%500 ==0:
            print("\n\n", cur_frame, "out of", total_num_frames)
        cur_frame +=1 

        ret, frame = cap.read()
        

        frame.flags.writeable = False
        #adjust orientation
        frame= prepare_frame(frame)


        frame_timestamp_ms = int(cur_frame/fpms)
        #print(ret)

        hand_dictionary = get_hand_landmarks(frame, video_detector, 
                                       timestamp = frame_timestamp_ms,
                                      display=False)

        transformed_hand_dct= compute_transformed_hand_points(hand_dictionary, 
                                                             warp_mat, frame, 
                                                             display=False)


        warp_dst = perform_warp(frame,warp_mat, display=False)
        
        


        '''
        Plot bounding box and save results depending on hand
        '''
        if 'Right' in transformed_hand_dct:
            
            #Draw bounds
            warp_dst = draw_notecloud(warp_dst, transformed_hand_dct, "Right", rh_cloud_color )

            #Draw hand points
            warp_dst = draw_handpoints(warp_dst, transformed_hand_dct,  "Right", obs_idx, rh_keypoint_color)


            #Save hand landmark data        
            rh_res.append(transformed_hand_dct['Right'])

        else:
            rh_res.append([])
            
            
        if 'Left' in transformed_hand_dct:
            
            # Draw bounds
            warp_dst = draw_notecloud(warp_dst, transformed_hand_dct, "Left", lh_cloud_color )

            # Draw handpoints
            warp_dst = draw_handpoints(warp_dst, transformed_hand_dct,  "Left", obs_idx, lh_keypoint_color)
            
            
            lh_res.append(transformed_hand_dct['Left'])      
        else:
            lh_res.append([])

        if display:
            warp_dst = perform_warp(frame,warp_mat, display=False)
            fig=plt.figure()

            plt.axvspan(rh_bounds[0], rh_bounds[1],  alpha=0.3, color='red')

            plt.axvspan(lh_bounds[0], lh_bounds[1],  alpha=0.3, color='blue')
            

            plt.imshow(warp_dst, origin='lower')        
            
            break

        if saveVid:
            #So vis will show up right
            warp_dst = cv2.flip(warp_dst, 0)
            #plt.imshow(warp_dst)
            #plt.show()
            #break

            #write to first video
            if cur_frame < boundary_frame:
                out1.write(warp_dst)
            
            #close first video, open second video, write first frame
            elif cur_frame == boundary_frame:
                out1.release()
                out2_pth = outfile_stem+"-2.avi"
                out2 = cv2.VideoWriter(out2_pth,
                            cv2.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width, frame_height))
                out2.write(warp_dst)
            else:
                out2.write(warp_dst)




    cap.release()
    if saveVid:

        out2.release()
        
    return rh_res, lh_res

# Extract warp and handpoints from all videos

Generates visualization of mediapipe landmarks on video, as well as two files in the form "_NOTECLOUD_.pkl" specifiying landmarks on right and left hand. 

## Read template


In [ ]:
if not local:
    test_img_file = "./small_assets/top_piano_piecestart.jpeg"
    template_FILE  = "./small_assets/template.png"

else:
    test_img_file ="../data/dataset/testing/top_piano_piecestart.jpeg"
    template_FILE  = "../data/dataset/testing/template.png"
template_color =  cv2.imread(cv2.samples.findFile(template_FILE  ))
template_color  = cv2.flip(template_color , 0)
T_HEIGHT = template_color.shape[0]
T_WIDTH = template_color.shape[1]

plt.imshow(template_color)

## Adjust Hard coded parameters ( position of middle fingers in the first frame)

`scorepoints`  - the locations of the middle fingers on the left and right hand on the reference frame. Unit is length of white key. The location of middle C is hardcoded, so these values can be specified relatively. 
`start` - Location (frames) in the video where reference frame is taken

In [ ]:
mf_points = {
#'B01':{'vidfile':'b-01.mp4', 'scorepoints': [18.5, 32.5], "start":0},
#'B02':{'vidfile':'b-02.mp4', 'scorepoints':[MIDDLE_C_POS, MIDDLE_C_POS+7], "start":0},#C4, C5
#'B03':{'vidfile':'b-03.mp4', 'scorepoints':[MIDDLE_C_POS-1, MIDDLE_C_POS+7], "start":0},#C4, C5
'B04':{'vidfile':'b-04.mp4', 'scorepoints':[MIDDLE_C_POS-2, MIDDLE_C_POS+8], "start":0},#C4, C5    
#'B05':{'vidfile':'b-05.mp4', 'scorepoints':[MIDDLE_C_POS-2, MIDDLE_C_POS+12], "start":0},#B05: A3 A5
#'B06':{'vidfile':'b-06.mp4', 'scorepoints':[MIDDLE_C_POS-5, MIDDLE_C_POS+4], "start":0},#B06: E3 G3
#'B07':{'vidfile':'b-07.mp4', 'scorepoints':[MIDDLE_C_POS-8.5, MIDDLE_C_POS+1.5], "start":0},#B06: E3 G3

#'B08':{'vidfile':'b-08.mp4', 'scorepoints':[MIDDLE_C_POS-2, MIDDLE_C_POS+7], "start":0},    
#'B10':{'vidfile':'b-10.mp4', 'scorepoints':[MIDDLE_C_POS-0, MIDDLE_C_POS+9], "start":5},
#'B11':{'vidfile':'b-11.mp4', 'scorepoints':[MIDDLE_C_POS+3, MIDDLE_C_POS+10], "start":5},
#'B13':{'vidfile':'b-13.mp4', 'scorepoints':[MIDDLE_C_POS-1, MIDDLE_C_POS+7.5], "start":0},
#'B14':{'vidfile':'b-14.mp4', 'scorepoints':[MIDDLE_C_POS-4.5, MIDDLE_C_POS+2.5], "start":4},
#'B15':{'vidfile':'b-15.mp4', 'scorepoints':[MIDDLE_C_POS-9, MIDDLE_C_POS+1.5], "start":4},
#'B20':{'vidfile':'b-20.mp4', 'scorepoints':[MIDDLE_C_POS-6, MIDDLE_C_POS+3], "start":0},
    
    # need to run ones below
#'B16':{'vidfile':'b-16.mp4', 'scorepoints':[MIDDLE_C_POS+0.5, MIDDLE_C_POS+8.5], "start":8},

#'B21':{'vidfile':'b-21.mp4', 'scorepoints':[MIDDLE_C_POS-.5, MIDDLE_C_POS+6], "start":0},
'B24':{'vidfile':'b-24.mp4', 'scorepoints':[MIDDLE_C_POS-7, MIDDLE_C_POS+4], "start":12},
#try different parse! 'B16':{'vidfile':'b-16.mp4', 'scorepoints':[MIDDLE_C_POS-1.5, MIDDLE_C_POS+6], "start":0},
 #try different parse! 'B19':{'vidfile':'b-19.mp4', 'scorepoints':[MIDDLE_C_POS-0.5, MIDDLE_C_POS+7.5], "start":0},
 #try different parse!'B20':{'vidfile':'b-20.mp4', 'scorepoints':[MIDDLE_C_POS-6, MIDDLE_C_POS+3.5], "start":4},
}

#B06: E3 G3
#B07: A#2 D#4
#B08: 

## Run fun pipeline

In [ ]:
display = True
home_dir = "../data/dataset/Burgmuller_top_kaitlin"
saveVid = True
for folder in mf_points:


        print("*******************\nFile", folder)

        vid_pth= os.path.join(home_dir, folder, mf_points[folder]['vidfile'])
        
        div_pth = os.path.join(home_dir, folder, 'div.txt')
        with open(div_pth) as f:
            contents = f.readlines()
            boundary_time = int(contents[0])
                  

        #get alignment frame
        refHandTime = mf_points[folder]["start"]
        hand_ref_frame, alignment_computation_frame, boundary_frame, fps= get_boundary_frame(
            vid_pth, boundary_time,refHandTime)

        


        scorepoints = mf_points[folder]['scorepoints']
        final_wrp = get_piano_alignment_transform(template_color=template_color, 
                                                  src_color=hand_ref_frame, 
                                                  alignment_computation_frame=alignment_computation_frame,
                                                  detector=detector, 
                                                  middle_finger_notes=scorepoints, 
                                                  display=True)

        
        
        
        initialized_prev_warp = final_wrp.copy()
        
        


        outfile_stem = os.path.join(home_dir, folder, "NOTECLOUD_"+folder)
        rh_res, lh_res = extract_hand_info_from_burgmuller(vid_pth, boundary_frame, 
                                          landmarker_model_pth, final_wrp,
                                          saveVid=saveVid, outfile_stem=outfile_stem)
        
        
        
        
        rh_res_pth =  os.path.join(home_dir, folder,"NOTECLOUD_"+folder+"-rh.pkl")
        lh_res_pth =  os.path.join(home_dir, folder,"NOTECLOUD_"+folder+"-lh.pkl")
        print("saved paths are", rh_res_pth, lh_res_pth)
        
        with open(rh_res_pth  , 'wb') as handle:
            pickle.dump({'boundary': boundary_frame, 'fps': fps, 'data':rh_res}, handle, protocol=pickle.HIGHEST_PROTOCOL) 
        
        
        with open(lh_res_pth  , 'wb') as handle:
            pickle.dump({'boundary': boundary_frame, 'fps': fps,'data':lh_res}, handle, protocol=pickle.HIGHEST_PROTOCOL)    
     

## Version where previously computed initialization is used (for takes with issues

In [ ]:
mf_points = {
#'B01':{'vidfile':'b-01.mp4', 'scorepoints': [18.5, 32.5], "start":0},
#'B02':{'vidfile':'b-02.mp4', 'scorepoints':[MIDDLE_C_POS, MIDDLE_C_POS+7], "start":0},#C4, C5
#'B03':{'vidfile':'b-03.mp4', 'scorepoints':[MIDDLE_C_POS-1, MIDDLE_C_POS+7], "start":0},#C4, C5
#'B04':{'vidfile':'b-04.mp4', 'scorepoints':[MIDDLE_C_POS-2, MIDDLE_C_POS+8], "start":0},#C4, C5    
#'B05':{'vidfile':'b-05.mp4', 'scorepoints':[MIDDLE_C_POS-2, MIDDLE_C_POS+12], "start":0},#B05: A3 A5
#'B06':{'vidfile':'b-06.mp4', 'scorepoints':[MIDDLE_C_POS-5, MIDDLE_C_POS+4], "start":0},#B06: E3 G3
#'B07':{'vidfile':'b-07.mp4', 'scorepoints':[MIDDLE_C_POS-8.5, MIDDLE_C_POS+1.5], "start":0},#B06: E3 G3

#'B08':{'vidfile':'b-08.mp4', 'scorepoints':[MIDDLE_C_POS-2, MIDDLE_C_POS+7], "start":0},    
#'B10':{'vidfile':'b-10.mp4', 'scorepoints':[MIDDLE_C_POS-0, MIDDLE_C_POS+9], "start":5},
#'B11':{'vidfile':'b-11.mp4', 'scorepoints':[MIDDLE_C_POS+3, MIDDLE_C_POS+10], "start":5},
#'B13':{'vidfile':'b-13.mp4', 'scorepoints':[MIDDLE_C_POS-1, MIDDLE_C_POS+7.5], "start":0},
#'B14':{'vidfile':'b-14.mp4', 'scorepoints':[MIDDLE_C_POS-4.5, MIDDLE_C_POS+2.5], "start":4},
#'B15':{'vidfile':'b-15.mp4', 'scorepoints':[MIDDLE_C_POS-9, MIDDLE_C_POS+1.5], "start":4},
#'B20':{'vidfile':'b-20.mp4', 'scorepoints':[MIDDLE_C_POS-6, MIDDLE_C_POS+3], "start":0},
#'B24':{'vidfile':'b-24.mp4', 'scorepoints':[MIDDLE_C_POS-7, MIDDLE_C_POS+4], "start":12},    
    # need to run ones below
#'B16':{'vidfile':'b-16.mp4', 'scorepoints':[MIDDLE_C_POS+0.5, MIDDLE_C_POS+8.5], "start":8},

'B21':{'vidfile':'b-21.mp4', 'scorepoints':[MIDDLE_C_POS-.5, MIDDLE_C_POS+6], "start":0},

#try different parse! 'B16':{'vidfile':'b-16.mp4', 'scorepoints':[MIDDLE_C_POS-1.5, MIDDLE_C_POS+6], "start":0},
'B19':{'vidfile':'b-19.mp4', 'scorepoints':[MIDDLE_C_POS-0.5, MIDDLE_C_POS+7.5], "start":0},
'B20':{'vidfile':'b-20.mp4', 'scorepoints':[MIDDLE_C_POS-6, MIDDLE_C_POS+3.5], "start":4},
}

#B06: E3 G3
#B07: A#2 D#4
#B08: 

In [ ]:
display = True
home_dir = "../data/dataset/Burgmuller_top_kaitlin"
saveVid = True
for folder in mf_points:


        print("*******************\nFile", folder)

        vid_pth= os.path.join(home_dir, folder, mf_points[folder]['vidfile'])
        
        div_pth = os.path.join(home_dir, folder, 'div.txt')
        with open(div_pth) as f:
            contents = f.readlines()
            boundary_time = int(contents[0])
                  

        #get alignment frame
        refHandTime = mf_points[folder]["start"]
        hand_ref_frame, alignment_computation_frame, boundary_frame, fps= get_boundary_frame(
            vid_pth, boundary_time,refHandTime)

        


        scorepoints = mf_points[folder]['scorepoints']
        '''
        final_wrp = get_piano_alignment_transform(template_color=template_color, 
                                                  src_color=hand_ref_frame, 
                                                  alignment_computation_frame=alignment_computation_frame,
                                                  detector=detector, 
                                                  middle_finger_notes=scorepoints, 
                                                  display=True, prevComputedInitialization=initialized_prev_warp)

        
        
        
        '''        
        
        


        outfile_stem = os.path.join(home_dir, folder, "NOTECLOUD_"+folder)
        rh_res, lh_res = extract_hand_info_from_burgmuller(vid_pth, boundary_frame, 
                                          landmarker_model_pth, final_wrp,
                                          saveVid=saveVid, outfile_stem=outfile_stem)
        
        
        
        
        rh_res_pth =  os.path.join(home_dir, folder,"NOTECLOUD_"+folder+"-rh.pkl")
        lh_res_pth =  os.path.join(home_dir, folder,"NOTECLOUD_"+folder+"-lh.pkl")
        print("saved paths are", rh_res_pth, lh_res_pth)
        
        with open(rh_res_pth  , 'wb') as handle:
            pickle.dump({'boundary': boundary_frame, 'fps': fps, 'data':rh_res}, handle, protocol=pickle.HIGHEST_PROTOCOL) 
        
        
        with open(lh_res_pth  , 'wb') as handle:
            pickle.dump({'boundary': boundary_frame, 'fps': fps,'data':lh_res}, handle, protocol=pickle.HIGHEST_PROTOCOL)    
     

# Tutorial: Extract single video info

## Constants

- path to video
- path to `div.txt` file specifying time (in seconds) that is boundary between take 1 and take 2 of the piece. This file needs to be manually created by looking at the video
- refHandTime: location (in frames) of the frame used for alignment


In [ ]:
vid_pth = '../data/dataset/Burgmuller_top_kaitlin/B02/b-02.mp4'
div_pth = '../data/dataset/Burgmuller_top_kaitlin/B02/div.txt'
refHandTime = 0 

## Extract and visualize alignment frame

Visually check that boundary frame is empty

In [ ]:
# extract boundary
with open(div_pth) as f:
    contents = f.readlines()
    boundary_time = int(contents[0])
                  

#get alignment frame
refHandTime = mf_points[folder]["start"]
hand_ref_frame, alignment_computation_frame, boundary_frame, fps= get_boundary_frame(
    vid_pth, boundary_time,refHandTime)

        

## Visually inspect the alginment frame for the positions of the two middle fingers in relation to middle C. Specify their location in terms of white notes away from Middle C

- New data has fixed reference position, but for this dataset finger locations must be manually inputted. 

In [ ]:
scorepoints = [MIDDLE_C_POS, MIDDLE_C_POS+7]

## Obtain warp matrix needed to shift alignment frame to predetermined template location

As a sanity check, make sure the notes specified by the red corners in the image match those in the template and that the mediapipe landmarks looks generally correct

In [ ]:

final_wrp = get_piano_alignment_transform(template_color=template_color, 
                                          src_color=hand_ref_frame, 
                                          alignment_computation_frame=alignment_computation_frame,
                                          detector=detector, 
                                          middle_finger_notes=scorepoints, 
                                          display=True)



## Extract hand landmark points. This is the raw data, postprocessing is performed in `FINAL PIPELINE 3`

In [ ]:
#path for videos
outfile_stem = os.path.join(home_dir, folder, "NOTECLOUD_"+folder)

#path for extracted landmarks
rh_res_pth =  os.path.join(home_dir, folder,"NOTECLOUD_"+folder+"-rh.pkl")
lh_res_pth =  os.path.join(home_dir, folder,"NOTECLOUD_"+folder+"-lh.pkl")
print("saved paths are", rh_res_pth, lh_res_pth)

In [ ]:
rh_res, lh_res = extract_hand_info_from_burgmuller(vid_pth, boundary_frame, 
                                  landmarker_model_pth, final_wrp,
                                  saveVid=saveVid, outfile_stem=outfile_stem)

with open(rh_res_pth  , 'wb') as handle:
    pickle.dump({'boundary': boundary_frame, 'fps': fps, 'data':rh_res}, handle, protocol=pickle.HIGHEST_PROTOCOL) 


with open(lh_res_pth  , 'wb') as handle:
    pickle.dump({'boundary': boundary_frame, 'fps': fps,'data':lh_res}, handle, protocol=pickle.HIGHEST_PROTOCOL)    
